# Lab Optimización - Programación Lineal

El TL del equipo de Advanced Analytics debe asignar las tareas del siguiente sprint entre los colaboradores de su equipo para cumplir con los objetivos propuestos. 

Entre las tareas se encuentran las _marcadas_ las cuales pertenecen a los objetivos y deben ser terminadas durante el sprint (10 días), las tareas no marcadas no tienen que ser terminadas.

En discuciones con negocio se puntuaron las tareas según prioridad e importancia, y se quiere que durante el sprint se obtenga la mayor puntuación posible.

Cada tarea puede ser ejecutada por un solo colaborador del equipo, y todos los colaboradores deben de trabajar al menos 8 días, y no puede trabajar mas de 10 días.

Cada día laboral consta de 8 horas.

## Generación de datos

Primero listamos las tareas y obtenemos el tiempo que tarda cada uno de los colaboradores en completar las mismas.


In [3]:
%%time
import generate_example as ge

tasks = ge.generate_tasks(100)
team_stats = ge.generate_team_stats(tasks)
# Los tiempos expresados estan en horas.

Wall time: 316 ms


## Modelo de programación lineal

### Datos

$N = $ Cantidad de tareas.

$T = $ Conjunto de todas las tareas.

$O = $ Conjunto de las tareas objetivo.

$C = $ Conjunto de todos los colaboradores.

$P_{t} = $ Puntaje de la tarea $t$.

$T_{t}^{c} = $ Tiempo que le toma al colaborador $c$ completar la tarea $t$ (horas).


### Variables

$X_{t}^{c}$ (binaria) Asignamos la tarea $t$ al colaborador $c$.

$Y_{t}$ (binaria) La tarea $t$ fue completada.

### Función objetivo

$$Max \sum_{t=1}^{N} P_{t}Y_{t}$$

### Restricciones

Tarea ejecutada por un solo colaborador:
$$\sum_{c \in C} X_{t}^{c} = Y_{t}, \forall t \in T$$

Cumplimiento de objetivos:
$$Y_{t} = 1, \forall t \in O$$

Horas trabajadas por cada colaborador:
$$64 \leq \sum_{t = 1}^{N} T_{t}^{c}X_{t}^{c} \leq 80, \forall c \in C$$


## Solución

In [4]:
%%time
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')

# Declaramos las variables.
asignacion = {f"{colaborador}_{task}" : solver.BoolVar(f"{colaborador}_{task}") for colaborador in team_stats.ID for task in tasks.ID}
ejecucion = {f"{task}" : solver.BoolVar(f"{task}") for task in tasks.ID}

# Declaramos la función objetivo
solver.Maximize(sum(task.points * ejecucion[f"{task.ID}"] for _,task in tasks.iterrows()))

# Declaramos las restricciones
## Tarea ejcutada por un solo colaborador
for task in tasks.ID:
    solver.Add(sum(asignacion[f"{colaborador}_{task}"] for colaborador in team_stats.ID) == ejecucion[f"{task}"])
    
## Cumplimiento de objetivos
for _,task in tasks.iterrows():
    if not task.objective:
        continue
    solver.Add(ejecucion[f"{task.ID}"] == 1)
    
# Horas trabajadas
for _,colaborador in team_stats.iterrows():
    solver.Add(sum(colaborador[f"time_to_task_{task}"] * asignacion[f"{colaborador.ID}_{task}"] for task in tasks.ID) >= 64)
    solver.Add(sum(colaborador[f"time_to_task_{task}"] * asignacion[f"{colaborador.ID}_{task}"] for task in tasks.ID) <= 80)

# Resolvemos
status = solver.Solve()

# Resulados
if status == pywraplp.Solver.OPTIMAL:
    print(f"Puntos completados: {solver.Objective().Value()}")
    print(f"Tareas completadas: {sum(var.solution_value() for var in ejecucion.values())}")
    for colaborador in team_stats.ID:
        print(f"\t- {colaborador}: {sum(var.solution_value() for var_name, var in asignacion.items() if colaborador in var_name)}")
else:
    print("No se encontro el optimo")

Puntos completados: 411.0
Tareas completadas: 58.0
	- IB: 3.0
	- ML: 5.0
	- GA: 8.0
	- SB: 8.0
	- FP: 4.0
	- MM: 1.0
	- YO: 3.0
	- DA: 8.0
	- FS: 6.0
	- AF: 2.0
	- GV: 10.0
Wall time: 4.74 s
